In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from keras.applications.densenet import DenseNet201, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD, adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard

In [ ]:
train_dir = '/datasets/tiny-imagenet-200/train'
val_dir = '/datasets/tiny-imagenet-200/val'

In [ ]:
im_width, im_height = 64, 64 #fixed size for DenseNet
num_label = 200
batch_size = 128
num_of_epochs = 10

## Transfer leraning from DenseNet

In [ ]:
# bliock = [6, 12, 48, 32]
base_model = DenseNet201(include_top=False)

In [ ]:
# Add new layer after DenseNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_label, activation='softmax', name='fc_predict')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze all weight in base network for transfer learning
for layer in base_model.layers:
    layer.trainable = False

opt = adam(lr=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Data Generator

In [ ]:
# TODO - validation# TODO  

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    # rotation_range=30,
    rescale=1/255,
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(im_width, im_height),
    batch_size=batch_size,
)

In [ ]:
# Plot learning curve on tensor board# Plot  
tensor_board = TensorBoard(log_dir='/log/tensorboard', batch_size=batch_size)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_generator, 
                    epochs = num_of_epochs,
                    # validation_data=val_generator,
                    callbacks = [tensor_board])